# import the necessary packages

In [1]:
import numpy as np
import argparse
import imutils
from imutils import paths
import cv2

![](https://www.pyimagesearch.com/wp-content/uploads/2018/12/image_stitching_opencv_ideal-1024x458.jpg)


We’ll be reviewing basic image processing operations including threshold, contour extraction, morphological operations, etc. in order to obtain our desired result.

# load our input images

In [2]:
# grab the paths to the input images and initialize our images list
img_path = "./scottsdale"
print("[INFO] loading images...") 
imagepath = sorted(list(paths.list_images(img_path))) # grab our imagePaths from img_path to format a list

# initialize our images list
images = []
# loop over the image paths, load each one, and add them to our images to stich list
for imagepath in imagepath:
    image = cv2.imread(imagepath)
    images.append(image)
# print(images)
print("[TNFO] load images finished")

[INFO] loading images...
[TNFO] load images finished


# initialize OpenCV's image sticher object and then perform the image stitching


In [3]:
print("[TNFO] stitching images ...")

stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)


[TNFO] stitching images ...


In [4]:
def show(img, name):
    cv2.imshow("%s" %(name), img)
    key = cv2.waitKey()
    if key == 27:
        cv2.destroyAllWindows()

In [5]:
crop = 1

# if the status is '0', then OpenCV successfully performed image stitching
if status == 0:
    # check to see if we supposed to crop out the largest rectangular region from the stitched image
    if crop > 0:
        # create a 10 pixel border surrounding the stitched image
        print("[INFO] cropping...")
        stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,cv2.BORDER_CONSTANT, (0, 0, 0))
        
        # convert the stitched image to grayscale and threshold it
        # such that all pixels greater than zero are set to 255
        # (foreground) while all others remain 0 (background)
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
        
        # find all external contours in the threshold image then find
        # the *largest* contour which will be the contour/outline of
        # the stitched image
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        
        # allocate memory for the mask which will contain the
        # rectangular bounding box of the stitched image region
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
        
        # create two copies of the mask: one to serve as our actual
        # minimum rectangular region and another to serve as a counter
        # for how many pixels need to be removed to form the minimum
        # rectangular region
        minRect = mask.copy()
        sub = mask.copy()

        # keep looping until there are no non-zero pixels left in the
        # subtracted image
        while cv2.countNonZero(sub) > 0:
            # erode the minimum rectangular mask and then subtract
            # the thresholded image from the minimum rectangular mask
            # so we can count if there are any non-zero pixels left
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)

        # find contours in the minimum rectangular mask and then
        # extract the bounding box (x, y)-coordinates
        cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        (x, y, w, h) = cv2.boundingRect(c)

        # use the bounding box coordinates to extract the our final stitched image
        stitched = stitched[y:y + h, x:x + w]

    # write the output stitched image to disk
    cv2.imwrite("./scottsdale/out_optim.png", stitched)

    # display the output stitched image to our screen
    show(stitched, name="./scottsdale/out_optim.png")

# otherwise the stitching failed, likely due to not enough keypoints)
# being detected
else:
    print("[ERROR] image stitching failed ({})".format(status))


[INFO] cropping...
